# Challenge: Data cleaning & validation

In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
df = pd.read_csv('WELLCOME_APCspend2013_forThinkful.csv', encoding='latin-1')
df.head(n=3)

,PMID/PMCID,Publisher,Journal title,Article title,COST (£) charged to Wellcome (inc VAT when charged)
0,NaN,CUP,Psychological Medicine,Reduced parahippocampal cortical thickness in ...,£0.00
1,PMC3679557,ACS,Biomacromolecules,Structural characterization of a Model Gram-ne...,£2381.04
2,23043264 PMC3506128,ACS,J Med Chem,"Fumaroylamino-4,5-epoxymorphinans and related ...",£642.56


## Clean up the dataset

In [3]:
# Fill empty cells:
df = df.fillna('n_a')

# Strip whitespace from beggining and end of strings in the entire dataframe:
df = df.apply(lambda x: x.str.strip()if x.dtype == "object" else x)

# Rename the Cost column:
df.rename(columns={'COST (£) charged to Wellcome (inc VAT when charged)':'Cost'}, inplace=True)

# Remove the currency symbol:
df['Cost'] = df['Cost'].str.replace('£', '')

# Convert Cost column to integer. Convert to NAN any values that aren't numbers:
df['Cost'] = pd.to_numeric(df['Cost'], errors='coerce')

## Determine the five most common journals and the total articles for each.

In [8]:
# Create a list with the names of the top five journals:
top_journals = df[['Journal title','Publisher']].groupby(
    ['Journal title']).count().sort_values('Publisher',ascending=False)
top_journals = top_journals.head(n=5)
top_journals = list(top_journals.index.values)

# Filter rows with values for the top 5 journals:
article_count = df[df['Journal title'].isin(top_journals)]

In [9]:
# Count values by Journal title:
article_count['Journal title'].value_counts()

PLoS One                                           92
PLoS ONE                                           62
Journal of Biological Chemistry                    48
Nucleic Acids Research                             24
Proceedings of the National Academy of Sciences    21
Name: Journal title, dtype: int64

## Calculate the mean, median, and standard deviation of the open-access cost per article for each journal.

In [10]:
# Replace values that are more than 4x the median and
# create a function that can be useful in the future:
def replace_outliers(col,factor):
    try:
        median_factor = col.median() * factor
    except TypeError:
        print('Column values must be numeric')
    else:
        new_col = col.apply(
            lambda x: median_factor if (x > median_factor) or x < (median_factor*-1)
            else x)
        return new_col

df['Cost_minus_outliers'] = replace_outliers(df['Cost'],4)

In [11]:
df.describe()

,Cost,Cost_minus_outliers
count,2114.000000,2114.000000
mean,24206.441892,1960.069139
std,147300.992149,1156.550910
min,0.000000,0.000000
25%,1280.000000,1280.000000
50%,1889.950000,1889.950000
75%,2322.315000,2322.315000
max,999999.000000,7559.800000


## For a real bonus round, identify the open access prices paid by subject area.

In [12]:
df['Subject area'] = df['Journal title'].str.lower()

In [19]:
replace_values = {
    'journal':'', 'j ':'', ' j':'', 'of':'', 'acs':'', 'society':'', 'reviews':'',
    'traffic':'', 'uk':'', 'american':'', 'plos one':'', 'plos':'', '\n':''}

df.replace({"Subject area": replace_values}, regex=True, inplace=True)

In [40]:
df[['Subject area','Cost_minus_outliers']].head(n=15)

,Subject area,Cost_minus_outliers
0,psychological medicine,0.00
1,biomacromolecules,2381.04
2,med chem,642.56
3,med chem,669.64
4,org chem,685.88
5,medicinal chemistry,2392.20
6,proteome research,2367.95
7,mol pharm,649.33
8,chemical biology,1294.59
9,chemical biology,1294.78
